In [ ]:
# ============================================
# Diamond Dynamics: Price Prediction and Market Segmentation
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import skew

# Data preprocessing and feature engineering
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Clustering
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# ANN
import tensorflow as tf
from tensorflow import keras
from keras import layers, Sequential
from keras.callbacks import EarlyStopping

# Pickle for saving models
import pickle
import joblib

# ============================================
# 1. DATA LOADING AND INITIAL INSPECTION
# ============================================

# Load dataset (replace with your actual file path)
def load_data(filepath='diamonds.csv'):
    """Load diamond dataset"""
    df = pd.read_csv(filepath)
    print(f"Dataset Shape: {df.shape}")
    print("\nFirst 5 rows:")
    print(df.head())
    print("\nDataset Info:")
    print(df.info())
    print("\nBasic Statistics:")
    print(df.describe())
    return df

# ============================================
# 2. DATA CLEANING AND PREPROCESSING
# ============================================

def clean_data(df):
    """Clean and preprocess the data"""
    
    # Create copy
    df_clean = df.copy()
    
    # Check for missing values
    print("Missing values before cleaning:")
    print(df_clean.isnull().sum())
    
    # Handle zero/invalid values in x, y, z
    for col in ['x', 'y', 'z']:
        # Replace zeros with NaN
        df_clean[col] = df_clean[col].replace(0, np.nan)
        # Replace with median based on carat
        df_clean[col] = df_clean.groupby(pd.cut(df_clean['carat'], 
                                                bins=[0, 0.5, 1, 1.5, 2, 3, 5]))[col].transform(
                                                    lambda x: x.fillna(x.median()))
    
    # Drop any remaining rows with NaN (if any)
    df_clean = df_clean.dropna()
    
    # Remove duplicates
    df_clean = df_clean.drop_duplicates()
    
    print(f"\nShape after cleaning: {df_clean.shape}")
    return df_clean

# ============================================
# 3. OUTLIER HANDLING
# ============================================

def handle_outliers(df):
    """Handle outliers using IQR method"""
    df_out = df.copy()
    
    numerical_cols = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
    
    for col in numerical_cols:
        Q1 = df_out[col].quantile(0.25)
        Q3 = df_out[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Cap outliers
        df_out[col] = np.where(df_out[col] < lower_bound, lower_bound, df_out[col])
        df_out[col] = np.where(df_out[col] > upper_bound, upper_bound, df_out[col])
    
    return df_out

# ============================================
# 4. SKEWNESS HANDLING
# ============================================

def handle_skewness(df):
    """Handle skewness in numerical features"""
    df_skew = df.copy()
    
    # Check skewness
    numerical_cols = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
    print("Skewness before transformation:")
    for col in numerical_cols:
        print(f"{col}: {df_skew[col].skew():.4f}")
    
    # Apply log transformation to highly skewed features
    skewed_cols = ['price', 'carat', 'x', 'y', 'z']
    for col in skewed_cols:
        if col in df_skew.columns:
            df_skew[f'{col}_log'] = np.log1p(df_skew[col])
    
    print("\nSkewness after log transformation (price_log):", 
          df_skew['price_log'].skew())
    
    return df_skew

# ============================================
# 5. EXPLORATORY DATA ANALYSIS (EDA)
# ============================================

def perform_eda(df):
    """Perform comprehensive EDA"""
    
    # Set style
    sns.set_style("whitegrid")
    
    # 1. Distribution plots
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    numerical_cols = ['carat', 'price', 'x', 'y', 'z', 'depth']
    for idx, col in enumerate(numerical_cols[:6]):
        sns.histplot(df[col], kde=True, ax=axes[idx])
        axes[idx].set_title(f'Distribution of {col}')
    
    plt.tight_layout()
    plt.savefig('distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 2. Count plots for categorical features
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    categorical_cols = ['cut', 'color', 'clarity']
    for idx, col in enumerate(categorical_cols):
        order = df[col].value_counts().index
        sns.countplot(data=df, x=col, ax=axes[idx], order=order)
        axes[idx].set_title(f'Distribution of {col}')
        axes[idx].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('categorical_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 3. Price variation with categorical features
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, col in enumerate(categorical_cols):
        sns.boxplot(data=df, x=col, y='price', ax=axes[idx])
        axes[idx].set_title(f'Price by {col}')
        axes[idx].tick_params(axis='x', rotation=45)
        axes[idx].set_yscale('log')
    
    plt.tight_layout()
    plt.savefig('price_vs_categorical.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 4. Correlation heatmap
    numerical_df = df.select_dtypes(include=[np.number])
    plt.figure(figsize=(10, 8))
    corr_matrix = numerical_df.corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                fmt='.2f', square=True)
    plt.title('Correlation Heatmap')
    plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 5. Scatterplot matrix
    sns.pairplot(df[['carat', 'price', 'x', 'y', 'z']], 
                diag_kind='kde', plot_kws={'alpha': 0.6})
    plt.suptitle('Pairwise Relationships', y=1.02)
    plt.savefig('pairplot.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 6. Carat vs Price
    plt.figure(figsize=(10, 6))
    sns.regplot(data=df, x='carat', y='price', scatter_kws={'alpha': 0.3}, 
               line_kws={'color': 'red'})
    plt.title('Carat vs Price (with Regression Line)')
    plt.xlabel('Carat')
    plt.ylabel('Price (USD)')
    plt.savefig('carat_vs_price.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 7. Average price per category
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, col in enumerate(categorical_cols):
        avg_price = df.groupby(col)['price'].mean().sort_values()
        avg_price.plot(kind='bar', ax=axes[idx])
        axes[idx].set_title(f'Average Price by {col}')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Average Price (USD)')
        axes[idx].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('avg_price_categories.png', dpi=300, bbox_inches='tight')
    plt.show()

# ============================================
# 6. FEATURE ENGINEERING
# ============================================

def feature_engineering(df):
    """Create new features"""
    df_fe = df.copy()
    
    # Convert price to INR (assuming 1 USD = 83 INR)
    conversion_rate = 83
    df_fe['price_inr'] = df_fe['price'] * conversion_rate
    
    # New engineered features
    df_fe['volume'] = df_fe['x'] * df_fe['y'] * df_fe['z']
    df_fe['price_per_carat'] = df_fe['price'] / df_fe['carat']
    df_fe['dimension_ratio'] = (df_fe['x'] + df_fe['y']) / (2 * df_fe['z'])
    
    # Carat categories
    conditions = [
        df_fe['carat'] < 0.5,
        (df_fe['carat'] >= 0.5) & (df_fe['carat'] <= 1.5),
        df_fe['carat'] > 1.5
    ]
    choices = ['Light', 'Medium', 'Heavy']
    df_fe['carat_category'] = np.select(conditions, choices, default='Medium')
    
    # Additional features
    df_fe['surface_area'] = 2 * (df_fe['x']*df_fe['y'] + df_fe['x']*df_fe['z'] + df_fe['y']*df_fe['z'])
    df_fe['density'] = df_fe['carat'] / df_fe['volume']
    
    # Interaction features
    df_fe['carat_cut_interaction'] = df_fe['carat'] * pd.Categorical(df_fe['cut']).codes
    df_fe['carat_color_interaction'] = df_fe['carat'] * pd.Categorical(df_fe['color']).codes
    
    print("New features created:")
    print(df_fe[['volume', 'price_per_carat', 'dimension_ratio', 
                'carat_category', 'price_inr']].head())
    
    return df_fe

# ============================================
# 7. FEATURE SELECTION
# ============================================

def select_features(df, target_col='price'):
    """Select most important features"""
    
    # Separate features and target
    X = df.drop(columns=[target_col, 'price_inr', 'price_log'], errors='ignore')
    y = df[target_col]
    
    # Encode categorical variables
    X_encoded = pd.get_dummies(X, columns=['cut', 'color', 'clarity', 'carat_category'], 
                              drop_first=True)
    
    # Method 1: Random Forest Feature Importance
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_encoded, y)
    
    feature_importance = pd.DataFrame({
        'feature': X_encoded.columns,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("Top 10 Features by Importance:")
    print(feature_importance.head(10))
    
    # Method 2: Correlation with target
    X_encoded['target'] = y
    correlation = X_encoded.corr()['target'].abs().sort_values(ascending=False)
    print("\nTop 10 Features by Correlation:")
    print(correlation[1:11])
    
    # Select top features (example: top 15)
    top_features = feature_importance.head(15)['feature'].tolist()
    
    return top_features, X_encoded[top_features]

# ============================================
# 8. ENCODING AND SCALING
# ============================================

def prepare_data(df, features, target_col='price'):
    """Prepare data for modeling"""
    
    # Select features
    X = df[features]
    y = df[target_col]
    
    # Identify categorical columns
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    
    # Create preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
        ])
    
    return X, y, preprocessor

# ============================================
# 9. REGRESSION MODELING
# ============================================

def build_regression_models(X_train, X_test, y_train, y_test, preprocessor):
    """Build and compare regression models"""
    
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Lasso Regression': Lasso(alpha=0.1),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
        'KNN': KNeighborsRegressor(n_neighbors=5)
    }
    
    results = []
    
    for name, model in models.items():
        # Create pipeline
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        # Train model
        pipeline.fit(X_train, y_train)
        
        # Predictions
        y_pred_train = pipeline.predict(X_train)
        y_pred_test = pipeline.predict(X_test)
        
        # Calculate metrics
        train_mae = mean_absolute_error(y_train, y_pred_train)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        
        results.append({
            'Model': name,
            'Train MAE': train_mae,
            'Test MAE': test_mae,
            'Train RMSE': train_rmse,
            'Test RMSE': test_rmse,
            'Train R²': train_r2,
            'Test R²': test_r2
        })
        
        # Save the best model (based on test R²)
        if name == 'Random Forest':  # Example: save Random Forest
            joblib.dump(pipeline, 'best_regression_model.pkl')
            print(f"\nSaved {name} model as 'best_regression_model.pkl'")
    
    results_df = pd.DataFrame(results)
    print("\nRegression Models Performance:")
    print(results_df.to_string())
    
    return results_df

# ============================================
# 10. ARTIFICIAL NEURAL NETWORK (ANN)
# ============================================

def build_ann_model(X_train, X_test, y_train, y_test, preprocessor):
    """Build ANN model"""
    
    # Preprocess data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # Define ANN architecture
    model = Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train_processed.shape[1],)),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)  # Output layer for regression
    ])
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    # Early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    # Train model
    history = model.fit(
        X_train_processed, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate model
    train_pred = model.predict(X_train_processed, verbose=0)
    test_pred = model.predict(X_test_processed, verbose=0)
    
    train_mae = mean_absolute_error(y_train, train_pred)
    test_mae = mean_absolute_error(y_test, test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    ann_results = pd.DataFrame([{
        'Model': 'ANN',
        'Train MAE': train_mae,
        'Test MAE': test_mae,
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Train R²': train_r2,
        'Test R²': test_r2
    }])
    
    print("\nANN Performance:")
    print(ann_results.to_string())
    
    # Save ANN model
    model.save('ann_diamond_model.h5')
    print("\nSaved ANN model as 'ann_diamond_model.h5'")
    
    # Plot training history
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(history.history['loss'], label='Training Loss')
    axes[0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0].set_title('Model Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    
    axes[1].plot(history.history['mae'], label='Training MAE')
    axes[1].plot(history.history['val_mae'], label='Validation MAE')
    axes[1].set_title('Model MAE')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].legend()
    
    plt.tight_layout()
    plt.savefig('ann_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return ann_results, model

# ============================================
# 11. CLUSTERING FOR MARKET SEGMENTATION
# ============================================

def perform_clustering(df):
    """Perform K-Means clustering for market segmentation"""
    
    # Prepare data for clustering (exclude price and target variables)
    clustering_features = ['carat', 'x', 'y', 'z', 'depth', 'table']
    categorical_features = ['cut', 'color', 'clarity']
    
    # Encode categorical variables
    df_cluster = df.copy()
    le_cut = LabelEncoder()
    le_color = LabelEncoder()
    le_clarity = LabelEncoder()
    
    df_cluster['cut_encoded'] = le_cut.fit_transform(df_cluster['cut'])
    df_cluster['color_encoded'] = le_color.fit_transform(df_cluster['color'])
    df_cluster['clarity_encoded'] = le_clarity.fit_transform(df_cluster['clarity'])
    
    # Combine all features for clustering
    cluster_cols = clustering_features + ['cut_encoded', 'color_encoded', 'clarity_encoded']
    X_cluster = df_cluster[cluster_cols]
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_cluster)
    
    # Determine optimal number of clusters using Elbow method
    inertia = []
    silhouette_scores = []
    K_range = range(2, 11)
    
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        inertia.append(kmeans.inertia_)
        
        if k > 1:  # Silhouette score requires at least 2 clusters
            silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    
    # Plot Elbow method and Silhouette scores
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(K_range, inertia, marker='o')
    axes[0].set_xlabel('Number of Clusters (k)')
    axes[0].set_ylabel('Inertia')
    axes[0].set_title('Elbow Method for Optimal k')
    axes[0].grid(True)
    
    axes[1].plot(list(K_range)[1:], silhouette_scores, marker='o', color='orange')
    axes[1].set_xlabel('Number of Clusters (k)')
    axes[1].set_ylabel('Silhouette Score')
    axes[1].set_title('Silhouette Scores')
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.savefig('clustering_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Choose optimal k (example: k=4)
    optimal_k = 4
    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    # Add cluster labels to dataframe
    df_cluster['cluster'] = clusters
    
    # Save the clustering model
    joblib.dump(kmeans, 'kmeans_clustering_model.pkl')
    joblib.dump(scaler, 'clustering_scaler.pkl')
    
    # PCA for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    df_cluster['pca1'] = X_pca[:, 0]
    df_cluster['pca2'] = X_pca[:, 1]
    
    # Visualize clusters
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(df_cluster['pca1'], df_cluster['pca2'], 
                         c=df_cluster['cluster'], cmap='viridis', alpha=0.6)
    plt.colorbar(scatter, label='Cluster')
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.title('Diamond Clusters (PCA Visualization)')
    plt.savefig('clusters_pca.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Analyze clusters
    cluster_analysis = df_cluster.groupby('cluster').agg({
        'price': ['mean', 'median', 'count'],
        'carat': ['mean', 'median'],
        'cut': lambda x: x.mode()[0],
        'color': lambda x: x.mode()[0],
        'clarity': lambda x: x.mode()[0]
    }).round(2)
    
    print("\nCluster Analysis:")
    print(cluster_analysis)
    
    # Create cluster names based on characteristics
    cluster_names = {}
    for cluster_num in range(optimal_k):
        cluster_data = df_cluster[df_cluster['cluster'] == cluster_num]
        
        avg_price = cluster_data['price'].mean()
        avg_carat = cluster_data['carat'].mean()
        common_cut = cluster_data['cut'].mode()[0]
        
        if avg_price > df_cluster['price'].quantile(0.75) and avg_carat > 1.0:
            name = "Premium Heavy Diamonds"
        elif avg_price < df_cluster['price'].quantile(0.25) and avg_carat < 0.5:
            name = "Affordable Small Diamonds"
        elif avg_price > df_cluster['price'].quantile(0.5) and common_cut in ['Premium', 'Ideal']:
            name = "High-Quality Premium Diamonds"
        else:
            name = "Mid-range Balanced Diamonds"
        
        cluster_names[cluster_num] = name
    
    print("\nCluster Names:")
    for cluster_num, name in cluster_names.items():
        print(f"Cluster {cluster_num}: {name}")
    
    # Save cluster names
    joblib.dump(cluster_names, 'cluster_names.pkl')
    
    return df_cluster, cluster_names, kmeans, scaler

# ============================================
# 12. MAIN EXECUTION PIPELINE
# ============================================

def main():
    """Main execution pipeline"""
    
    print("=" * 60)
    print("DIAMOND DYNAMICS: Price Prediction & Market Segmentation")
    print("=" * 60)
    
    # Step 1: Load data
    print("\n1. LOADING DATA...")
    df = load_data()
    
    # Step 2: Data cleaning
    print("\n2. DATA CLEANING...")
    df_clean = clean_data(df)
    
    # Step 3: Handle outliers
    print("\n3. HANDLING OUTLIERS...")
    df_outliers = handle_outliers(df_clean)
    
    # Step 4: Handle skewness
    print("\n4. HANDLING SKEWNESS...")
    df_skew = handle_skewness(df_outliers)
    
    # Step 5: Perform EDA
    print("\n5. EXPLORATORY DATA ANALYSIS...")
    perform_eda(df_skew)
    
    # Step 6: Feature engineering
    print("\n6. FEATURE ENGINEERING...")
    df_fe = feature_engineering(df_skew)
    
    # Step 7: Feature selection
    print("\n7. FEATURE SELECTION...")
    top_features, X_selected = select_features(df_fe)
    
    # Step 8: Prepare data for modeling
    print("\n8. PREPARING DATA FOR MODELING...")
    X, y, preprocessor = prepare_data(df_fe, top_features + ['cut', 'color', 'clarity', 'carat_category'])
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Step 9: Build regression models
    print("\n9. BUILDING REGRESSION MODELS...")
    regression_results = build_regression_models(
        X_train, X_test, y_train, y_test, preprocessor
    )
    
    # Step 10: Build ANN model
    print("\n10. BUILDING ANN MODEL...")
    ann_results, ann_model = build_ann_model(
        X_train, X_test, y_train, y_test, preprocessor
    )
    
    # Step 11: Perform clustering
    print("\n11. PERFORMING CLUSTERING...")
    df_clustered, cluster_names, kmeans_model, cluster_scaler = perform_clustering(df_fe)
    
    print("\n" + "=" * 60)
    print("MODEL TRAINING COMPLETED!")
    print("=" * 60)
    print("\nModels saved:")
    print("1. Best Regression Model: best_regression_model.pkl")
    print("2. ANN Model: ann_diamond_model.h5")
    print("3. Clustering Model: kmeans_clustering_model.pkl")
    print("4. Cluster Names: cluster_names.pkl")
    print("5. Clustering Scaler: clustering_scaler.pkl")
      